## Sample scripts for using crosswalk json files

In [1]:
import pandas as pd
from pandas import read_excel
import json
import os
from datetime import datetime

In [15]:
## Invert directionality of mapping
def invert_mappings(property_list):
    inverted_prop_list = []
    for i in range(len(property_list)):
        eachprop = property_list.iloc[i]
        newprop = eachprop.pop('sameAs')
        newprop['sameAs'] = eachprop
        inverted_prop_list.append(newprop)
    return inverted_prop_list

## Create mappings
def create_nestednames_map(property_list):
    for i in range(len(property_list)):
        eachprop = property_list.iloc[i]
        nestednames_map[eachprop['nestedName']] = eachprop['sameAs']['nestedName']
    return nestednames_map

## Convert a property based on schema
def convert_prop_name(property_list,a_property):
    nestednames_map = create_nestednames_map(property_list)
    converted_prop_name = nestednames_map[a_property]
    return converted_prop_name

def get_converted_prop_info(property_list,converted_prop_name):
    inverted_prop_list = invert_mappings(property_list)
    prop_info = [x for x in inverted_prop_list if x['nestedName']==converted_prop_name]
    return prop_info

In [3]:
script_path = ''

data_path = os.path.join(script_path,'jsoncrosswalks')
data_files = os.listdir(data_path)
#data_file = os.path.join(data_path,'nde_clinepidb_dataset.json')
#schema_path = os.path.join(script_path,'schema')
#schema_file = os.path.join(schema_path,'MetadataCrossWalk.json')

In [22]:
## read the files

property_list = pd.DataFrame(columns=['nestedName','isPartOf','name','sameAs'])
for data_file in data_files:
    with open(os.path.join(data_path,data_file),'r') as jsonfile:
        crosswalkmeta = json.load(jsonfile)
        tmpproplist = crosswalkmeta['includesProperty']
        tmpdf = pd.DataFrame(tmpproplist)
        property_list = pd.concat((property_list,tmpdf),ignore_index=True)

property_list.fillna('unspecified',inplace=True)
print(len(property_list))

446


In [24]:
## Get properties and equivalencies
prop2find = "topicCategory.name"

resultdf = property_list.loc[(property_list['nestedName']==prop2find)|(property_list['name']==prop2find)]
print(len(resultdf))
#print(resultdf)
nde_subset = resultdf.loc[resultdf['isPartOf']=={'@id':'nde:Dataset'}]
equivalences = pd.DataFrame(nde_subset['sameAs'].tolist()).fillna('unspecified')
print(equivalences)

2
               nestedName owl:cardinality  marginality  \
0  infectiousDisease.name            many  unspecified   

                    domainIncludes             rangeIncludes  \
0  [{'@id': 'schema:DefinedTerm'}]  [{'@id': 'schema:Text'}]   

                       url                     isPartOf  name  
0  https://schema.org/name  {'@id': 'outbreak:Dataset'}  name  


In [19]:
## Convert a property using the mapping
converted_prop_name = convert_prop_name(property_list,'attributes.description')
print(converted_prop_name,'\n')

description 



In [20]:

## Pull info on the mapped property
converted_prop_info = get_converted_prop_info(property_list,converted_prop_name)
print(converted_prop_info)

[{'nestedName': 'description', 'owl:cardinality': 'one', 'marginality': 'minimal or required', 'domainIncludes': [{'@id': 'nde:ComputationalTool'}], 'rangeIncludes': 'schema:Text', 'url': 'https://schema.org/description', 'isPartOf': {'@id': 'nde:ComputationalTool'}, 'name': 'description', 'sameAs': nestedName                     Description
isPartOf           {'@id': 'bioconductor'}
name                           Description
owl:cardinality                        one
Name: 1, dtype: object}]
